#Import

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD

import os
import random
import collections
from collections import defaultdict

import shutil
from shutil import copy
from shutil import copytree, rmtree

import numpy as np
from google.colab import files

import matplotlib.pyplot as plt
import matplotlib.image as img
import matplotlib.image as mpimg
%matplotlib inline

#Download Datasets

In [ ]:
def get_dataset_and_extract():
  if "food-101" in os.listdir():
    print("Dataset already exists")
  else:
    tf.keras.utils.get_file(
      'food-101.tar.gz',
      'http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz',
      cache_subdir='/content',
      extract=True,
      archive_format='tar',
      cache_dir=None
    )

In [ ]:
get_dataset_and_extract()

# Inspect the Dataset Structure

##Folder Structure and Files

In [ ]:
main_path = 'food-101/'

In [ ]:
os.listdir(main_path)

In [ ]:
os.listdir(os.path.join(main_path, 'images'))

In [ ]:
os.listdir(os.path.join(main_path, 'meta'))

In [ ]:
classes_files = os.path.join(main_path, 'meta/classes.txt')

classes = []
with open(classes_files, 'r') as txt:
    classes = [read.strip() for read in txt.readlines()]

classes

In [ ]:
labels_files = os.path.join(main_path, 'meta/labels.txt')

labels = []
with open(labels_files, 'r') as txt:
    labels = [read.strip() for read in txt.readlines()]

labels

##Visualize image

In [ ]:
# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

print(classes)

rdm_classes = random.sample(classes, len(classes))[:4]     

pic_index+=4

rdm_classes_img = []
for class_item in rdm_classes: 
  class_img = [os.path.join(main_path+'/images/'+class_item, fname) 
                      for fname in os.listdir(main_path+'/images/'+class_item)[pic_index-4:pic_index]
                    ]
  rdm_classes_img.extend(class_img)

for i, img_path in enumerate(rdm_classes_img):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

#Split the image data into train and validation using train.txt and test.txt

In [ ]:
def split_dataset(filepath, src, dir):
  images_files = defaultdict(list)
  with open(filepath, 'r') as txt:
      paths = [read.strip() for read in txt.readlines()]
      for p in paths:
        images_path = p.split('/')
        images_files[images_path[0]].append(images_path[1] + '.jpg')

  for food in images_files.keys():
    if not os.path.exists(os.path.join(dir,food)):
      os.makedirs(os.path.join(dir,food))
    for i in images_files[food]:
      copy(os.path.join(src,food,i), os.path.join(dir,food,i))

In [ ]:
train_dir = main_path+'datasets/train'
validation_dir = main_path+'datasets/validation'

In [ ]:
# Prepare train dataset
split_dataset(main_path+'meta/train.txt', main_path+'images', train_dir)

In [ ]:
# Prepare validation dataset
split_dataset(main_path+'meta/test.txt', main_path+'images', validation_dir)

In [ ]:
# Check how many files are in the train folder
train_files = sum([len(files) for i, j, files in os.walk(train_dir)])
print("Total number of files in train folder")
print(train_files)

In [ ]:
# Check how many files are in the test folder
validation_files = sum([len(files) for i, j, files in os.walk(validation_dir)])
print("Total number of files in validation folder")
print(validation_files)

#Training and Validation Generators

In [ ]:
def train_val_generator(train_dir, validation_dir, batch_size, img_width, img_height):
  train_datagen = ImageDataGenerator(
      rescale = 1./255.,
      rotation_range = 40,
      width_shift_range = 0.2,
      height_shift_range = 0.2,
      shear_range = 0.2,
      zoom_range = 0.2,
      horizontal_flip = True)

  validation_datagen = ImageDataGenerator(rescale = 1./255.)

  train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size
  )

  validation_generator = validation_datagen.flow_from_directory(
      validation_dir,
      target_size=(img_height, img_width),
      batch_size=batch_size,
      class_mode='categorical')
  
  return train_generator, validation_generator


In [ ]:
batch_size = 128
img_width = 300
img_height = 300

In [ ]:
train_generator, validation_generator = train_val_generator(train_dir, validation_dir, batch_size, img_width, img_height)

#Create Model

In [ ]:
# Download the inception v3 weights
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
# Create an instance of the inception model from the local pre-trained weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
def create_pre_trained_model(local_weights_file, img_width, img_height):
  pre_trained_model = InceptionV3(input_shape = (img_width, img_height, 3),
                                  include_top = False, 
                                  weights = None) 

  pre_trained_model.load_weights(local_weights_file)

  # Make all the layers in the pre-trained model non-trainable
  for layer in pre_trained_model.layers:
    layer.trainable = False

  return pre_trained_model

In [ ]:
pre_trained_model = create_pre_trained_model(local_weights_file, img_width, img_height)

# Print the model summary
pre_trained_model.summary()

In [ ]:
total_params = pre_trained_model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in pre_trained_model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

In [ ]:
def output_of_last_layer(pre_trained_model):
  last_desired_layer = pre_trained_model.get_layer('mixed7')
  print('last layer output shape: ', last_desired_layer.output_shape)
  last_output = last_desired_layer.output
  print('last layer output: ', last_output)

  return last_output

In [ ]:
last_output = output_of_last_layer(pre_trained_model)

In [ ]:
num_classes = len(labels)
num_classes

In [ ]:
def create_model(pre_trained_model, last_output, num_classes):
  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)

  # Add a fully connected layer with 1024 hidden units and ReLU activation
  x = layers.Dense(512, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.2)(x)  
  # Add a final softmax layer for classification
  x = layers.Dense(num_classes, activation='softmax')(x)        

  # Create the complete model by using the Model class
  model = Model(inputs=pre_trained_model.input, outputs=x)

  # Compile the model
  model.compile(optimizer = SGD(learning_rate=0.0001), 
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [ ]:
# Save your model in a variable
model = create_model(pre_trained_model, last_output, num_classes)

# Inspect parameters
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

#Train Model

In [ ]:
num_epochs = 100

In [ ]:
def train_model(num_epochs):
  checkpoint_path = "training_1/cp.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)

  # Create a callback that saves the model's weights
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1)

  history = model.fit(train_generator,
                      steps_per_epoch=train_files // batch_size, 
                      validation_data=validation_generator,
                      validation_steps=train_files // batch_size,
                      epochs=num_epochs,
                      verbose=1,
                      callbacks=[cp_callback])
  return history

In [ ]:
history = train_model(num_epochs)

#Visualize the accuracy and loss plots

In [ ]:
def plot_accuracy(history,title):
    plt.title(title)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()


def plot_loss(history,title):
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()


plot_accuracy(history,'Model Accuracy')
plot_loss(history,'Model Loss')

#Predicting

In [ ]:
def predict_class(model, images):
  for img in images:
    img = image.load_img(img, target_size=(img_width, img_height))
    x = image.img_to_array(img)                    
    x = np.expand_dims(img, axis=0)

    images = np.vstack([x])
    images = images / 255.                                   

    pred = model.predict(images)
    index = np.argmax(pred)
    pred_value = labels[index]
    print(pred)

    plt.imshow(img)                           
    plt.axis('off')
    plt.title(pred_value)
    plt.show()

In [ ]:
images = []

images.append(imagepath + 'hamburger/72111.jpg')
images.append(imagepath + 'fried_rice/1391.jpg')
images.append(imagepath + 'pizza/53217.jpg')

predict_class(model, images)

In [ ]:
# Predicting with random images
images = []
imagepath = main_path + 'images/'

for i in range(3):
  rdm_class = random.choice(classes)
  images.append(imagepath+rdm_class+'/'+random.choice(os.listdir(imagepath+rdm_class)))

predict_class(model, images)

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(img_width, img_height))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  images = images / 255.
  classes_pred = model.predict(images, batch_size=10)
  index = np.argmax(classes_pred)
  pred_value = labels[index]
  print(fn)
  print(classes_pred)

  plt.imshow(img)                           
  plt.axis('off')
  plt.title(pred_value)
  plt.show()

#Export the Model

In [ ]:
# Use the tf.saved_model API to save your model in the SavedModel format. 
export_dir = 'saved_model/1'

In [ ]:
tf.saved_model.save(model, export_dir)

In [ ]:
%%bash -s $export_dir
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

In [ ]:
loaded = tf.saved_model.load(export_dir)

In [ ]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

#save model to TFlite

In [ ]:
# Select mode of optimization
mode = None

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [ ]:
# Use the TFLiteConverter SavedModel API to initialize the converter
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
tflite_model = converter.convert()
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

# Test the TFLite Model Using the Python Interpreter

In [ ]:
# Load TFLite model and allocate tensors.
with open(tflite_model_file, 'rb') as fid:
    tflite_model = fid.read()
    
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [ ]:
def predict_tflite(images):
  for img_path in images:
    img = image.load_img(img_path, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    images = images / 255.                             

    interpreter.set_tensor(input_index, images)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_index)

    index = np.argmax(pred)
    pred_value = labels[index]
    print(pred)

    plt.imshow(img)                           
    plt.axis('off')
    plt.title(pred_value)
    plt.show()

In [ ]:
images = []

images.append(imagepath + 'hamburger/72111.jpg')
images.append(imagepath + 'fried_rice/1391.jpg')
images.append(imagepath + 'pizza/53217.jpg')

predict_tflite(images)

In [ ]:
# Gather results for the randomly sampled test images
images = []
imagepath = main_path + 'images/'

for i in range(3):
  rdm_class = random.choice(classes)
  images.append(imagepath+rdm_class+'/'+random.choice(os.listdir(imagepath+rdm_class)))

predict_tflite(images)

#Download Save Model and TF Lite

In [ ]:
!zip -r saved_model.zip saved_model

In [ ]:
try:
    files.download('converted_model.tflite')
    files.download('saved_model.zip')
    files.download(main_path+'meta/labels.txt')
except:
    pass